In [1]:
import os
import shutil
import json
import requests
import fitz
import nltk
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from chromadb.config import Settings
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


# Environment setup

In [2]:
def setup_environment():
    import sys
    sys.path.append('C:\\gitworkspace\\aimldemo\\jupyterworkapce')
    import stratup_env_setup
    stratup_env_setup.set_env()

In [3]:
setup_environment()

In [4]:
class HuggingFaceEmbeddings:
    """Custom wrapper for Hugging Face SentenceTransformer embeddings."""
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.embedding_model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts):
        """Generate embeddings for a list of documents."""
        return self.embedding_model.encode(texts, convert_to_tensor=False).tolist()
    
    def embed_query(self, text):
        """Generate embedding for a single query."""
        return self.embedding_model.encode(text, convert_to_tensor=False).tolist()

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
chunk_size=500
chunk_overlap=50

In [7]:
#pdf_path = "C:\\gitworkspace\\aimldemo\\AIML Demo\\Python Demo\\pythonbasics\\doc_inputs\\Form_2287.pdf"
pdf_folder = "C:\\gitworkspace\\aimldemo\\AIML Demo\\Python Demo\\pythonbasics\\doc_inputs"
output_folder = "C:\\gitworkspace\\aimldemo\\AIML Demo\\Python Demo\\pythonbasics\\doc_outputs"

In [8]:
highlighted_pdf_path = os.path.join(output_folder, "highlighted_output.pdf")

In [9]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

In [10]:
CREDIT_REVIEW_API_URL = "http://127.0.0.1:8000/getCreditReviewData"

# Vectorize PDF Document

In [11]:
db_path = "chroma_db"

In [12]:
if os.path.exists(db_path):
        print(f"[INFO] Deleting existing Chroma database at {db_path} to avoid conflicts.")
        shutil.rmtree(db_path, ignore_errors=True)

In [13]:
embeddings = HuggingFaceEmbeddings()

In [14]:
chroma_settings = Settings(persist_directory=db_path, anonymized_telemetry=False)

In [15]:
vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings, client_settings=chroma_settings)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_3300\1200157228.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings, client_settings=chroma_settings)


In [16]:
def extract_chunks_with_coordinates(pdf_path):
    """Extract chunks from the PDF along with their coordinates (bounding boxes) and file name."""
    doc = fitz.open(pdf_path)
    chunks_with_metadata = []

    # Extract the file name from the full path
    file_name = os.path.basename(pdf_path)

    for page_num in range(len(doc)):
        page = doc[page_num]
        page_text = page.get_text("blocks")
        full_text = " ".join(block[4] for block in page_text)

        splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        text_chunks = splitter.split_text(full_text)

        for chunk in text_chunks:
            coordinates = []
            search_results = page.search_for(chunk)
            for rect in search_results:
                coordinates.append((rect.x0, rect.y0, rect.x1, rect.y1))

            chunks_with_metadata.append({
                "text": chunk,
                "page": page_num,
                "coordinates": coordinates,
                "file_name": file_name  # Add file name to metadata
            })

    return chunks_with_metadata


In [17]:
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

In [18]:
pdf_files

['C:\\gitworkspace\\aimldemo\\AIML Demo\\Python Demo\\pythonbasics\\doc_inputs\\Form_2287.pdf',
 'C:\\gitworkspace\\aimldemo\\AIML Demo\\Python Demo\\pythonbasics\\doc_inputs\\lenovo_license_agreement.pdf']

In [19]:
text_chunks_with_metadata = []
for pdf_file in pdf_files:
    chunks = extract_chunks_with_coordinates(pdf_file)
    text_chunks_with_metadata.extend(chunks)

In [20]:
text_chunks_with_metadata[0]

{'text': 'SBA Form 2287 (04-18) \n \n          1 \n Previous Editions Obsolete \n  \n \n  \n  \n  \n THIRD PARTY LENDER AGREEMENT \n  \n \n \nTHIS THIRD PARTY LENDER AGREEMENT (“Agreement") is dated this ______ day \nof______________, 20____, by and between ________________________________________, \n(“Third Party Lender”) whose address is ____________________________________________, \nand________________________________________________________, (“CDC”) whose address is',
 'page': 0,
 'coordinates': [(74.28336334228516,
   736.0769653320312,
   161.34852600097656,
   747.1239013671875),
  (287.99462890625, 736.0769653320312, 290.229736328125, 747.1239013671875),
  (395.9959411621094, 736.0769653320312, 424.5137939453125, 747.1239013671875),
  (72.0, 745.435546875, 172.51608276367188, 756.4824829101562),
  (141.0, 82.16398620605469, 144.0, 98.17198944091797),
  (177.0, 82.16398620605469, 180.0, 98.17198944091797),
  (138.72000122070312,
   107.96397399902344,
   141.72000122070312,
   

In [21]:
page_number = 2
file_name = 'Form_2287.pdf'

In [22]:
for chunk in text_chunks_with_metadata:
    if chunk['file_name'] == file_name and chunk['page'] == page_number:
        print("Text Chunk:", chunk['text'])
        print("Page Number:", chunk['page'])
        print("File Name:", chunk['file_name'])
        print("#" * 50)

Text Chunk: SBA Form 2287 (04-18) 
 
          3 
 Previous Editions Obsolete 
 failure to pay taxes when due or violation of  any financial covenants which would cause a prudent 
lender to believe that the  prospect of payment or performance of the Third Party Note is impaired. 
 
 
c. 
No Cross-Collateralization.   Third Party Lender agrees that the Common Collateral 
 will only secure its Third Party Loan and the Common Collateral is not currently, and will not be
Page Number: 2
File Name: Form_2287.pdf
##################################################
Text Chunk: used in the future, as security for any other financing provided by Third Party Lender to Borrower 
that purports to be in a superior position to that of the CDC Lien, unless authorized in writing by 
CDC and SBA. 
 
 
d. 
No Cross-Default.   During the term of the 504 Loan, Third Party Lender will not 
 exercise any cross-default, "deem at-risk," or any other provisions in documents evidencing the Third
Page Number: 2
Fi

In [23]:
chunk_texts = [item["text"] for item in text_chunks_with_metadata]
metadata = [{"page_number": item["page"], "file_name": item["file_name"], "coordinates": json.dumps(item["coordinates"])} for item in text_chunks_with_metadata]

In [24]:
chunk_texts[0]

'SBA Form 2287 (04-18) \n \n          1 \n Previous Editions Obsolete \n  \n \n  \n  \n  \n THIRD PARTY LENDER AGREEMENT \n  \n \n \nTHIS THIRD PARTY LENDER AGREEMENT (“Agreement") is dated this ______ day \nof______________, 20____, by and between ________________________________________, \n(“Third Party Lender”) whose address is ____________________________________________, \nand________________________________________________________, (“CDC”) whose address is'

In [25]:
metadata[0]

{'page_number': 0,
 'file_name': 'Form_2287.pdf',
 'coordinates': '[[74.28336334228516, 736.0769653320312, 161.34852600097656, 747.1239013671875], [287.99462890625, 736.0769653320312, 290.229736328125, 747.1239013671875], [395.9959411621094, 736.0769653320312, 424.5137939453125, 747.1239013671875], [72.0, 745.435546875, 172.51608276367188, 756.4824829101562], [141.0, 82.16398620605469, 144.0, 98.17198944091797], [177.0, 82.16398620605469, 180.0, 98.17198944091797], [138.72000122070312, 107.96397399902344, 141.72000122070312, 123.97197723388672], [72.0, 133.7639617919922, 75.0, 149.77195739746094], [306.0, 159.56394958496094, 309.0, 175.5719451904297], [155.0399932861328, 184.8782196044922, 168.40667724609375, 212.6336212158203], [168.36000061035156, 189.1377410888672, 214.11094665527344, 211.24234008789062], [214.0800018310547, 184.8782196044922, 226.32444763183594, 212.6336212158203], [226.32000732421875, 189.1377410888672, 275.67633056640625, 211.24234008789062], [275.760009765625, 1

In [26]:
vector_db.add_texts(chunk_texts, metadatas=metadata)

['19a713b3-66a2-4230-b98d-61483fee3e69',
 '481bf2fe-c854-4ab6-9641-12920444a730',
 '4cee1a01-bb6b-45f9-a850-8e07dbc51cd4',
 '7a959c22-a8d0-445c-808d-5e4d9861f59f',
 'c616d640-b719-4b6b-9a58-72a748392d48',
 '955290c3-85fc-475a-bbc6-49fc5f4625b0',
 '9060f204-ffcd-47f3-9855-adc531008f87',
 '0cba957f-4e4a-462d-b28c-80f4e36feec3',
 '52d1b10f-c259-41b0-b488-5ee0178af1ae',
 'e7f8fcea-0637-4f96-8b29-5dd7db91d2a6',
 '3d81d1aa-5041-4f0c-a025-d17e06218b09',
 '7df3a73b-3513-46a5-9c21-e86f439c1eb7',
 '556b442f-88e9-4e6b-bf58-2c398817088f',
 '396814e3-4e71-4107-978d-befa83e8178f',
 '14798d55-510b-4583-ab9e-5fcdf0817bb0',
 'b598a32b-d15c-411f-ba93-d2364738ce08',
 '66f496d9-eb29-4860-81da-d86e3a1e2b73',
 '60ae8ba2-77e7-40c5-9765-fbe7c3e9ab56',
 'a510c1b0-9247-4c13-a8e7-54e9a3464714',
 '71968625-2c24-4517-ba99-893ad6b41df1',
 'f9e99764-966c-41cf-a29d-2020e84b2146',
 'a1b9b672-cffd-428a-bc78-f89aef02f9ee',
 '53cc6721-cb0f-4129-9c68-0cd060cd2406',
 '885ed0fd-a1ae-42bb-9642-2b33a5fe62e9',
 '7cebddd3-7425-

In [27]:
vector_db.persist()

C:\Users\jimmy\AppData\Local\Temp\ipykernel_3300\2766552738.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


# Chat Query

In [28]:
#user_query = 'What does this document say about Balloon payments?'
#user_query = "what does this document say about authority to execute agreement?"
#user_query = 'About Balloon payments in page_number 3 of Form_2287.pdf?'
#user_query = 'What does the docuemnt say about Transferability in page_number 3 of lenovo_license_agreement.pdf?'
user_query = 'How many credit reviews are there for cagid 123?'

In [29]:
chat_history = [
    {"role": "user", "content": "CDC agrees to make the 504 Loan to the Borrower subject to what?"},
    {"role": "assistant", "content": "SBA's approval."},
    {"role": "user", "content": "Third Party Lender waives its rights for what?"},
    {"role": "assistant", "content": "Third Party Lender waives its rights to enforce provisions that do not comply with the 504 Loan Program Requirements."},
    {"role": "user", "content": "what does this document say about authority to execute agreement?"},
    {"role": "assistant", "content": "The persons signing below certify that they have been duly authorized to execute this Agreement on behalf of their respective party."}
]

* Rephrase the query to identify teh user's intended full question using LLM

In [30]:
# Template to rephrase the user query and extract metadata
template = """
Given the chat history and the current user query, rewrite the query so it forms a complete question, maintaining the context of the conversation. Then, extract metadata (only "file_name" and "page_number" fields) if they are mentioned in the question or chat history.
Rewritten query shoudl not have refernce to metadata fields file_name and page_number.

Chat History:
{chat_history}

Current user query: {query}

Rewritten query:
[Provide the rewritten query here.]

Extracted Metadata (JSON format with only file_name and page_number fields):
[Provide the metadata here as a valid JSON object.]
"""

In [31]:
# Format the chat history into a string
formatted_chat_history = "\n".join([f'{entry["role"].capitalize()}: {entry["content"]}' for entry in chat_history])

In [32]:
# Create the prompt for the LLM
prompt = PromptTemplate.from_template(template).format(
    chat_history=formatted_chat_history,
    query=user_query
)

In [33]:
prompt

'\nGiven the chat history and the current user query, rewrite the query so it forms a complete question, maintaining the context of the conversation. Then, extract metadata (only "file_name" and "page_number" fields) if they are mentioned in the question or chat history.\nRewritten query shoudl not have refernce to metadata fields file_name and page_number.\n\nChat History:\nUser: CDC agrees to make the 504 Loan to the Borrower subject to what?\nAssistant: SBA\'s approval.\nUser: Third Party Lender waives its rights for what?\nAssistant: Third Party Lender waives its rights to enforce provisions that do not comply with the 504 Loan Program Requirements.\nUser: what does this document say about authority to execute agreement?\nAssistant: The persons signing below certify that they have been duly authorized to execute this Agreement on behalf of their respective party.\n\nCurrent user query: How many credit reviews are there for cagid 123?\n\nRewritten query:\n[Provide the rewritten quer

In [34]:
response = llm.predict(prompt)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_3300\403075660.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


In [35]:
response

'Rewritten query: How many credit reviews are there for cagid 123?\n\nExtracted Metadata:\n{\n  "file_name": "N/A",\n  "page_number": "N/A"\n}'

In [36]:
# Split the response into rewritten query and metadata parts
response_parts = response.split("\nExtracted Metadata:")

In [37]:
query = response_parts[0].replace("Rewritten query:", "").strip()

In [38]:
metadata_str = response_parts[1].strip()

In [39]:
metadata = json.loads(metadata_str)

In [40]:
metadata

{'file_name': 'N/A', 'page_number': 'N/A'}

In [41]:
query

'How many credit reviews are there for cagid 123?'

# Agent for Rest Services

In [42]:
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
    Analyze the following user query and determine whether to query the credit review REST API or search the vector database.

    If the query is related to credit review (e.g., approvals, CAGID, or review type), determine:
    - Mandatory parameters: CAGID
    - Optional filters: creationDate, creditReviewType, approvedBy, tfa
    - Which specific field the user wants (e.g., "approvedBy").

    Return a JSON object with the following fields:
    - "invoke_rest": true/false
    - "params": {{"CAGID": "<value>", "creationDate": "<value>", "creditReviewType": "<value>", "approvedBy": "<value>", "tfa": "<value>"}}
    - "required_field": the specific field the user needs, if applicable.

    User query: "{query}"
    """
)

In [43]:
formatted_prompt = prompt_template.format(query=user_query)

In [44]:
response = llm.predict(formatted_prompt)

In [45]:
response

'Based on the user query, it is related to credit review and specifically mentions the CAGID parameter. Therefore, we should query the credit review REST API.\n\nJSON object:\n{\n  "invoke_rest": true,\n  "params": {\n    "CAGID": "123"\n  },\n  "required_field": ""\n}'

In [46]:
import re

In [47]:
match = re.search(r'\{.*\}', response, re.DOTALL)
if match:
    json_str = match.group(0)
    try:
        response_dict = json.loads(json_str)
        print("Parsed JSON successfully:", response_dict)
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
else:
    print("No JSON object found in response.")

Parsed JSON successfully: {'invoke_rest': True, 'params': {'CAGID': '123'}, 'required_field': ''}


In [55]:
if response_dict.get("invoke_rest"):
    print("Calling REST API with parameters:", response_dict["params"])
else:
    print("No REST API call needed.")

Calling REST API with parameters: {'CAGID': '123'}


# If Condition to call REST Endpoint

In [56]:
api_params = {
    "cagid": response_dict["params"].get("CAGID")  # Convert key to lowercase
}

In [57]:
api_params

{'cagid': '123'}

In [58]:
response = requests.get(CREDIT_REVIEW_API_URL, params=api_params)

In [59]:
response

<Response [200]>

In [60]:
if response.status_code == 200:
    api_response = response.json()
    print("API Response Received Successfully:")
    print(json.dumps(api_response, indent=4))
else:
    print(f"API call failed with status code: {response.status_code}")
    print(response.text)

API Response Received Successfully:
[
    {
        "cagid": "123",
        "creationDate": "2024-01-15",
        "creditReviewType": "ANNUAL_REVIEW",
        "camId": "CAM001",
        "approvedBy": "SOEID001",
        "tfa": 5000000.0
    },
    {
        "cagid": "123",
        "creationDate": "2024-02-10",
        "creditReviewType": "INTERIM_REVIEW",
        "camId": "CAM002",
        "approvedBy": "SOEID002",
        "tfa": 2500000.0
    },
    {
        "cagid": "123",
        "creationDate": "2023-12-15",
        "creditReviewType": "ANNUAL_REVIEW",
        "camId": "CAM005",
        "approvedBy": "SOEID001",
        "tfa": 6000000.0
    }
]


In [61]:
prompt = f"""
You are an intelligent assistant. I have the following JSON data retrieved from a REST API:

JSON Structure:
- Each element in the JSON array represents a credit review record.
- Fields:
    - `cagid`: The unique identifier for the customer.
    - `creationDate`: The date the credit review was created.
    - `creditReviewType`: The type of credit review (e.g., ANNUAL_REVIEW, INTERIM_REVIEW, CREDIT_MEMO).
    - `camId`: The unique identifier for the credit application memo.
    - `approvedBy`: The SOEID of the approver.
    - `tfa`: The total facility amount for the credit review.

JSON Data:
{json.dumps(api_response, indent=4)}

Using the JSON data above, answer the following query in natural language:
{user_query}
"""

In [62]:
response = llm.predict(prompt)

In [63]:
response

'There are a total of 3 credit reviews for cagid 123 in the provided JSON data.'

# Fetch relevant chunks from Vector DB- (Else condition)

In [ ]:
 metadata_filter = {
        "$and": [
            {key: {"$eq": (value - 1 if key == "page_number" else value)}}
            for key, value in metadata.items()
        ]
    }

In [ ]:
metadata_filter

In [ ]:
top_chunks_with_scores = vector_db.similarity_search_with_score(query, k=3)
top_chunks_with_scores = vector_db.similarity_search_with_score(
    query=query,
    k=3,
    filter=metadata_filter
)

In [ ]:
top_chunks_with_scores

In [ ]:
# Extract chunk texts for cross-encoding
top_chunks = [chunk.page_content for chunk, _ in top_chunks_with_scores]

In [ ]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [ ]:
query_chunk_pairs = [(query, chunk) for chunk in top_chunks]

In [ ]:
query_chunk_pairs

In [ ]:
cross_encoded_scores = cross_encoder.predict(query_chunk_pairs)

In [ ]:
# Pair chunks with their cross-encoder scores and sort them in descending order
ranked_chunks_with_scores = sorted(
    zip(top_chunks_with_scores, cross_encoded_scores),
    key=lambda x: x[1],  # Sort by score (descending)
    reverse=True
)

In [ ]:
top_chunk_cross_encoded, top_cross_encoded_score = ranked_chunks_with_scores[0][0]

In [ ]:
metadata_of_top_chunk = top_chunk_cross_encoded.metadata

In [ ]:
metadata_of_top_chunk

In [ ]:
top_chunk_cross_encoded

In [ ]:
metadata_of_top_chunk

# LLM query

In [ ]:
llm_instruction = "Please answer the question by carefully considering both the provided context and the chat history. Use the context for accurate information and take into account any relevant details from the chat history to generate a well-informed response. If enough information is not available, please respond by saying: 'There is not enough information to answer the question.'"

In [ ]:
# Create the prompt
template = """
Context: 
{context}

Chat history:
{chat_history}

Instruction: 
{llm_instruction}

User question: {question}
"""

In [ ]:
formatted_prompt = PromptTemplate.from_template(template).format(
    context=top_chunk_cross_encoded.page_content,
    chat_history="\n".join([f'{entry["role"].capitalize()}: {entry["content"]}' for entry in chat_history]),
    question=query,
    llm_instruction=llm_instruction
)

In [ ]:
formatted_prompt

In [ ]:
# Get response from LLM
llm_response = llm.predict(formatted_prompt)

In [ ]:
print("#"*50)
print(llm_response)

# PDF Highlighting

In [ ]:
# Configurable cosine similarity threshold
COSINE_SIMILARITY_THRESHOLD = 0.5

In [ ]:
page_number =  metadata_of_top_chunk["page_number"]

In [ ]:
page_number

In [ ]:
coordinates_str = metadata_of_top_chunk["coordinates"]

In [ ]:
coordinates_str

In [ ]:
coordinates = json.loads(coordinates_str)

In [ ]:
coordinates

In [ ]:
file_for_highlighting = os.path.join(pdf_folder, metadata_of_top_chunk['file_name'])

In [ ]:
file_for_highlighting

In [ ]:
highlighted_pdf_path

In [ ]:
doc = fitz.open(file_for_highlighting)

In [ ]:
page = doc.load_page(page_num)

In [ ]:
type(page) 

In [ ]:
page.get_text()

In [ ]:
chunk_text = " ".join(page.get_textbox(fitz.Rect(*coord)) for coord in coordinates)

In [ ]:
chunk_text

In [ ]:
sentences = nltk.sent_tokenize(chunk_text)

In [ ]:
# Initialize Sentence Transformer for cosine similarity
similarity_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Compute embeddings for the LLM response and sentences
llm_response_embedding = similarity_model.encode(llm_response, convert_to_tensor=True)
sentence_embeddings = similarity_model.encode(sentences, convert_to_tensor=True)

In [ ]:
# Compute cosine similarities
similarities = util.cos_sim(llm_response_embedding, sentence_embeddings)[0].tolist()

In [ ]:
# Pair sentences with their similarity scores
sentences_with_scores = list(zip(sentences, similarities))

In [ ]:
# Sort sentences by similarity score in descending order
sorted_sentences_with_scores = sorted(sentences_with_scores, key=lambda x: x[1], reverse=True)

In [ ]:
for sentence, score in sorted_sentences_with_scores:
    print(f"Score: {score:.4f}, Sentence: {sentence}")
    print("#"*20)

In [ ]:
# Find high-similarity sentences and their coordinates
high_similarity_sentences = []
for sentence, score in sorted_sentences_with_scores:
    if score >= COSINE_SIMILARITY_THRESHOLD:
        search_results = page.search_for(sentence)
        if search_results:  # Ensure search_results is not empty
            high_similarity_sentences.append(fitz.Rect(*search_results[0]))

In [ ]:
high_similarity_sentences

In [ ]:
# Highlight the entire chunk in yellow
for coord in coordinates:
    rect = fitz.Rect(*coord)
    page.add_highlight_annot(rect)

In [ ]:
# Highlight high-similarity sentences in green
for rect in high_similarity_sentences:
    highlight = page.add_highlight_annot(rect)
    highlight.set_colors(stroke=(0, 1, 0))  # Green color
    highlight.update()

In [ ]:
doc.save(highlighted_pdf_path, garbage=4)
doc.close()